In [22]:
import pandas as pd
import numpy as np
import os
import ast
#from joblib import Parallel, delayed
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import ast

from datetime import date, datetime

In [2]:
# Recovering misssing financial data from data scraped from the-numbers.com

In [3]:
missing = pd.read_csv('data/missing_financial_data.csv')

In [4]:
all_movies = pd.read_csv('data/movies_metadata.csv')

/Users/shahidhn/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
the_numbers = pd.read_csv('data/the_numbers_data.csv')

In [6]:
all_ids = all_movies['imdb_id'].tolist()
missing_names = [all_movies['title'][all_ids.index(id)] for id in missing['recovered_ids']]
missing['title'] = missing_names

In [7]:
the_numbers_names = the_numbers['name'].tolist()

In [8]:
to_add = pd.DataFrame(columns=missing.dtypes.index.tolist())

# Name, budget, revenue (worldwide gross), imdb_id
for title in missing_names:
    if title in the_numbers_names:
        numbers_i = the_numbers_names.index(title)
        budget = the_numbers.iloc[numbers_i]['budget']
        revenue = the_numbers.iloc[numbers_i]['worldwide']
        if budget > 0 and revenue > 0:
            '''
            temp = []

            temp.append(the_numbers.iloc[numbers_i]['name'])
            temp.append(the_numbers.iloc[numbers_i]['budget'])
            temp.append(the_numbers.iloc[numbers_i]['worldwide'])
            '''

            missing_i = missing_names.index(title)
            add_row = missing.iloc[missing_i]
            add_row['budget'] = budget
            add_row['revenue'] = revenue
            to_add = to_add.append(add_row)

/Users/shahidhn/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/shahidhn/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
to_add.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,adult,budget,genres,id,original_language,production_countries,revenue,...,spoken_languages,movieId,imdbId,tmdbId,views,likes,dislikes,n_trailers,recovered_ids,title
2,2,2,2,False,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,en,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",87100000,...,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",7,114319,11860.0,878901,1,0,2,tt0114319,Sabrina
6,6,6,6,False,20000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",1710,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",32051917,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",22,112722,1710.0,315798,358,16,1,tt0112722,Copycat
8,8,8,8,False,11000000,"[{'id': 18, 'name': 'Drama'}]",16420,en,"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2844379,...,"[{'iso_639_1': 'en', 'name': 'English'}]",26,114057,16420.0,312163,413,29,1,tt0114057,Othello
14,14,14,14,False,13700000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",9603,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",56598476,...,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",39,112697,9603.0,3926883,6402,266,1,tt0112697,Clueless
15,15,15,15,False,9200000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",31174,en,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",4199334,...,"[{'iso_639_1': 'en', 'name': 'English'}]",41,114279,31174.0,192568,368,15,1,tt0114279,Richard III


In [62]:
# to_add.to_csv(path_or_buf='data/the_numbers_recovered.csv', mode='w+')

In [14]:
# Adding date features

date_df = pd.read_csv(os.path.join('data', 'movies_metadata.csv'))[['imdb_id', 'release_date']]

In [15]:
date_df.head()

,imdb_id,release_date
0,tt0114709,1995-10-30
1,tt0113497,1995-12-15
2,tt0113228,1995-12-22
3,tt0114885,1995-12-22
4,tt0113041,1995-02-10


In [18]:
to_add_dates = []
for id in to_add['recovered_ids'].tolist():
    i = date_df['imdb_id'].tolist().index(id)
    to_add_dates.append(date_df.iloc[i].release_date)

In [33]:
to_add['release_date'] = to_add_dates

In [39]:
epoch = datetime(1915, 2, 8)
daysSinceEpoch = []

dates = []
for strdate in to_add_dates:
    dates.append(datetime.strptime(strdate, '%Y-%m-%d'))
    
for date in dates:
    daysSinceEpoch.append((date - epoch).days)

to_add['daysSinceStart'] = daysSinceEpoch

In [44]:
dayInYear = []
for mdate in dates:
    dayInYear.append((mdate - datetime(mdate.year, 1, 1)).days)

to_add['dayInYear']= dayInYear

In [46]:
to_add.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,adult,budget,genres,id,original_language,production_countries,revenue,...,tmdbId,views,likes,dislikes,n_trailers,recovered_ids,title,release_date,daysSinceStart,dayInYear
2,2,2,2,False,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,en,"[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",87100000,...,11860.0,878901,1,0,2,tt0114319,Sabrina,1995-12-15,29530,348
6,6,6,6,False,20000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",1710,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",32051917,...,1710.0,315798,358,16,1,tt0112722,Copycat,1995-10-27,29481,299
8,8,8,8,False,11000000,"[{'id': 18, 'name': 'Drama'}]",16420,en,"[{'iso_3166_1': 'IT', 'name': 'Italy'}]",2844379,...,16420.0,312163,413,29,1,tt0114057,Othello,1995-12-15,29530,348
14,14,14,14,False,13700000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",9603,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",56598476,...,9603.0,3926883,6402,266,1,tt0112697,Clueless,1995-07-19,29381,199
15,15,15,15,False,9200000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",31174,en,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",4199334,...,31174.0,192568,368,15,1,tt0114279,Richard III,1995-12-29,29544,362
